In [1]:
import pandas as pd

## Load data

In [58]:
cleaned_deduped_authors_filename = "article_authors-2021-04-14-joined-authors_cleaned-deduped.csv"
newly_parsed_authors_filename = "magazine_authors_parsed.csv"
newly_merged_authors_filename = "magazine_authors_newly_merged.csv"

In [59]:

cleaned_deduped_authors = pd.read_csv(cleaned_deduped_authors_filename, skiprows=1)

In [60]:
cleaned_deduped_authors.head()

,drupal_full_name,drupal_author_id,duplicate of ID,given_name,family_name,organization_name,meeting_name,Notes
0,Carl Abbott,1352,NaN,Carl,Abbott,NaN,NaN,NaN
1,Margery Post Abbott,343,NaN,Margery Post,Abbott,NaN,NaN,NaN
2,Marge Abbott,2931,343.0,Margery Post,Abbott,NaN,NaN,NaN
3,Margery Abbott,3267,343.0,Margery Post,Abbott,NaN,NaN,NaN
4,Margery Post Abbott,3430,343.0,Margery Post,Abbott,NaN,NaN,NaN


In [61]:
cleaned_deduped_authors.shape

(2692, 8)

In [62]:
newly_parsed_authors = pd.read_csv(newly_parsed_authors_filename)

In [63]:
newly_parsed_authors.head()

,given_name,family_name,drupal_full_name,drupal_author_id
0,A. N.,Whitehead,A. N. Whitehead,1832
1,A.,S.,A. S.,1597
2,A. Sidney,Wright,A. Sidney Wright,1789
3,A. Stanley,Thompson,A. Stanley Thompson,2801
4,A.F.,Anderson,A.F. Anderson,2615


In [64]:
newly_parsed_authors.shape

(2774, 4)

## Merge new authors into existing spreadsheet

We will use the Drupal ID to skip existing rows and add only new authors to the merged spreadsheet.

In [66]:
# select newly parsed authors that are NOT in the cleaned deduped authors
new_authors = newly_parsed_authors[~newly_parsed_authors["drupal_author_id"].isin(cleaned_deduped_authors["drupal_author_id"])]

In [76]:
newly_merged_authors_for_export = pd.concat([cleaned_deduped_authors, new_authors])

In [80]:
newly_merged_authors_for_export.sort_values(by="drupal_full_name", inplace=True)

## Export

In [81]:
newly_merged_authors_for_export.to_csv(newly_merged_authors_filename, index=False)